In [1]:
import geocoder

geocode_res = geocoder.osm('1 Apple Park Way, Cupertino, California, United States')

In [2]:
coordinates = geocode_res.geometry['coordinates']
print(coordinates)

[-122.010747, 37.3317424]


In [3]:
import pandas as pd

df = pd.read_csv("ex09_geolocation_moving_obj.csv", names=['cab', 'lat', 'long', 'tm'])
display(df)

,cab,lat,long,tm
0,cab_26,43.602508,39.715685,14:47:44
1,cab_112,43.582243,39.752077,14:47:55
2,cab_26,43.607480,39.721521,14:49:11
3,cab_112,43.579258,39.758944,14:49:51
4,cab_112,43.574906,39.766325,14:51:53
5,cab_26,43.612203,39.720491,14:52:48


In [4]:
latestrows = df.sort_values(['cab', 'tm'], ascending=False).drop_duplicates('cab')
display(latestrows)

,cab,lat,long,tm
5,cab_26,43.612203,39.720491,14:52:48
4,cab_112,43.574906,39.766325,14:51:53


In [5]:
latestrows_numpy = latestrows.values
latestrows = latestrows_numpy.tolist()
display(latestrows)

[['cab_26', 43.612203, 39.720491, '14:52:48'],
 ['cab_112', 43.574906, 39.766325, '14:51:53']]

In [6]:
# calculate the distance between each cab and a pickup place
from geopy.distance import distance

pick_up = (43.578854, 39.754995)

for i, row in enumerate(latestrows):
    lat = row[1]
    long = row[2]
    cab = (lat, long)
    dist_meters = distance(pick_up, cab).m

    print(row[0] + ':', round(dist_meters))
    latestrows[i].append(round(dist_meters))

display(latestrows)

cab_26: 4636
cab_112: 1015


[['cab_26', 43.612203, 39.720491, '14:52:48', 4636],
 ['cab_112', 43.574906, 39.766325, '14:51:53', 1015]]

In [7]:
closest_cab = min(latestrows, key=lambda x: x[4])
print('The closest cab is: ', closest_cab[0], '. Distance in meters: ', closest_cab[4])

The closest cab is:  cab_112 . Distance in meters:  1015


Not always the closest object to the target is the most efficient solution. Some obstacles may infer in the time elapsed to reach the objective.

If obstacles makes traject to the objective longer an alternative is to divide the area into a set of polygons, and then use these polygons to check which objects share a polygon with the target.

In [8]:
from shapely.geometry import Point, Polygon

coords = [
    (46.082991, 38.987384),
    (46.075489, 38.987599),
    (46.079395, 38.997684),
    (46.073822, 39.007297),
    (46.081741, 39.008842)
]
poly = Polygon(coords)
cab_26 = Point(46.073852, 38.991890)
cab_112 = Point(46.078228, 39.003949)
pick_up = Point(46.080074, 38.991289)

print('cab_26 within the polygon:', cab_26.within(poly))
print('cab_112 within the polygon:', cab_112.within(poly))
print('pick_up within the polygon:', pick_up.within(poly))

cab_26 within the polygon: False
cab_112 within the polygon: True
pick_up within the polygon: True


The best approach is to use both calculations:

1. Determine which polygon each cab is in and use that determination to decide how to calculate the distance from that cab to the pick-up location. Then proceed to:
   1. Calculate direct straight line distance if the cab is in the same polygon as the pick-up location.
   2. The distance by way of the entry point if it’s in an adjacent polygon.

In [10]:
entry_point = Point(46.075357, 39.000298)

if cab_26.within(poly):
    dist = distance((pick_up.x, pick_up.y), (cab_26.x,cab_26.y)).m
else:
    dist = distance((cab_26.x,cab_26.y), (entry_point.x,entry_point.y)).m + distance((entry_point.x,entry_point.y), (pick_up.x, pick_up.y)).m

print(round(dist))

1544


In [13]:
orders = [
    ('order_039', 'open', 'cab_14'),
    ('order_034', 'open', 'cab_79'),
    ('order_032', 'open', 'cab_104'),
    ('order_026', 'closed', 'cab_79'),
    ('order_021', 'open', 'cab_45'),
    ('order_018', 'closed', 'cab_26'),
    ('order_008', 'closed', 'cab_112')
]

df_orders = pd.DataFrame(orders, columns =['order','status','cab'])
df_orders_open = df_orders[df_orders['status']=='open']
unavailable_list = [x[2] for x in orders if x[1] == 'open']
display(unavailable_list)

['cab_14', 'cab_79', 'cab_104', 'cab_45']

In [14]:
pick_up = 46.083822, 38.967845
cab_26 = 46.073852, 38.991890
cab_112 = 46.078228, 39.003949
cab_104 = 46.071226, 39.004947
cab_14 = 46.004859, 38.095825
cab_79 = 46.088621, 39.033929
cab_45 = 46.141225, 39.124934
cabs = {'cab_26': cab_26, 'cab_112': cab_112, 'cab_14': cab_14,
        'cab_104': cab_104, 'cab_79': cab_79, 'cab_45': cab_45}

dist_list = []

for cab_name, cab_loc in cabs.items():
  if cab_name not in unavailable_list:
    dist = distance(pick_up, cab_loc).m
    dist_list.append((cab_name, round(dist)))

print(dist_list)
print(min(dist_list, key=lambda x: x[1]))

[('cab_26', 2165), ('cab_112', 2861)]
('cab_26', 2165)


In [15]:
# Cabs data with Baby Seat
cabs_list = [
    ('cab_14',1),
    ('cab_79',0),
    ('cab_104',0),
    ('cab_45',1),
    ('cab_26',0),
    ('cab_112',1)
]

In [16]:
df_cabs = pd.DataFrame(cabs_list, columns =['cab', 'seat'])
df_dist = pd.DataFrame(dist_list, columns =['cab', 'dist'])

In [17]:
display(df_cabs)
display(df_dist)

,cab,seat
0,cab_14,1
1,cab_79,0
2,cab_104,0
3,cab_45,1
4,cab_26,0
5,cab_112,1


,cab,dist
0,cab_26,2165
1,cab_112,2861


In [18]:
df = pd.merge(df_cabs, df_dist, on='cab', how='inner')

In [19]:
display(df)

,cab,seat,dist
0,cab_26,0,2165
1,cab_112,1,2861


In [20]:
result_list = list(df.itertuples(index=False,name=None))
result_list = [x for x in result_list if x[1] == 1]
print(result_list)

[('cab_112', 1, 2861)]
